In [15]:
%load_ext autoreload
%autoreload 2

import pathlib
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(str(pathlib.Path().resolve().parent / "src"))
import constants

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data